In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [ ]:
# TO disable notifications
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

In [ ]:
driver = webdriver.Chrome(options=option,executable_path = r'C:\Program Files\chromedriver.exe')

In [ ]:
driver.get('https://instagram.com/')

# Credentials
username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "username")))
password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "password")))
username.clear()
password.clear()
username.send_keys("sample")          # Enter Username here
password.send_keys("sample")          # Enter Password here

btn = driver.find_elements_by_tag_name("button")
btn[1].click()

In [ ]:
driver.get('https://instagram.com/')
search_input =    driver.find_element_by_xpath("//input[@placeholder='Search']")
search_input.clear()
search_input.send_keys("fashion")
user_ids = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='fuqBx']//a")))


store_user_ids = []
for user_id in user_ids:
    html_content = user_id.get_attribute('outerHTML')
    data = BeautifulSoup(html_content,'html.parser').a['href'].split('/')[1]
    if data!='explore':
        store_user_ids.append(data)

for handle in store_user_ids:
    print(handle)

In [ ]:
driver.get('https://instagram.com')
search_input = driver.find_element_by_xpath("//input[@placeholder='Search']")
search_input.clear()
search_input.send_keys("allthefeels.official")

user_id = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//div[@class="fuqBx"]//a')))
user_id.click()

In [ ]:
# Follow
follow_button = driver.find_element_by_xpath("//span[@class = 'vBF20 _1OSdk']/button")

type_button = BeautifulSoup(follow_button.get_attribute('outerHTML'),'html.parser')
if type_button.text == 'Following':
    print('Already Following')
else:
    follow_button.click()
    print("Now Following")

In [ ]:
# Unfollow
type_button = BeautifulSoup(follow_button.get_attribute('outerHTML'),'html.parser')
if type_button.text!='Follow':
    follow_button.click()
    time.sleep(2) 
    unfollow = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//button[@class="aOOlW -Cab_   "]')))
    unfollow.click()
    print("Now Unfollowed")
else:
    print('Already Unfollowed')

In [ ]:
# Now Unfollowed
def like_unlike_post(choose):
    driver.get('https://instagram.com')
    search_input = driver.find_element_by_xpath("//input[@placeholder='Search']")
    search_input.clear()
    search_input.send_keys("haseenabyashna")
    time.sleep(3)
    user_id = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//div[@class="fuqBx"]//a')))
    user_id.click()
    time.sleep(2)

    post = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='v1Nh3 kIKUG  _bz0w']")))
    post.click()
    time.sleep(3)
    i = 1
    while True:
        like_button = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//article/div/section/span/button[@class='wpO6b ']")))
        type_button = BeautifulSoup(like_button.get_attribute('innerHTML'),'html.parser').find('svg').get('aria-label')
        
        if type_button.strip().lower()==choose.strip().lower():
            like_button.click()
        else:
            print('Already {}d'.format(choose))
        time.sleep(2)
        next_post = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div/a[@class=' _65Bje  coreSpriteRightPaginationArrow']")))
        next_post.click()
        i+=1
        time.sleep(3)
        if i==31:
            print("{}d 30 recent posts".format(choose.title()))
            driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//div[@class='_2dDPU CkGkG']/div/button[@class='wpO6b ']"))
            break

In [ ]:
# Like 30 Posts of allthefeels
like_unlike_post("like")
# Unlike  30 Posts of allthefeels
like_unlike_post("unlike")

In [ ]:
"""Here I have made one function that takes two arguments. First argument is for
what we are looking i.e following or follower, second argument is the user handle for which 
we want to see. There are four list and each one of them has separate work. 
The first two store the 500 followers of mentioned user_handles, and other two are
for own followers and following user_handles which is helping to decide which follower
from the haseenabyashna follower we are following and that follower doesn't follow back.
"""
def get_user_ids(is_following,user_handle):
    driver.get('https://instagram.com')
    search_input = driver.find_element_by_xpath("//input[@placeholder='Search']")
    search_input.clear()
    search_input.send_keys(user_handle)
    time.sleep(3)
    user_id = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//div[@class="fuqBx"]//a')))
    user_id.click()
    time.sleep(2)
    try:
        follower_button = driver.find_element_by_partial_link_text(is_following)
    except:
        return []
    time.sleep(2)
    follower_button.click()
    time.sleep(2)
    
    current_height = 0
    i = 0
    while True:
        try:
            prev_height = current_height
            current_height = driver.execute_script('return document.querySelector(".isgrP").scrollHeight;')
            driver.execute_script('document.querySelector(".isgrP").scrollTo(0,arguments[0]);',current_height)
            time.sleep(1)
            if prev_height==current_height:
                break
            i+=5
            if i==205:
                break
        except:
            break

    time.sleep(2)
    user_ids = []
    followers = driver.find_elements_by_class_name('d7ByH')


    for user_id in followers[:500]:
        user_id = BeautifulSoup(user_id.find_element_by_tag_name('a').get_attribute('outerHTML'),'html.parser')
        user_ids.append(user_id.a.get('title'))

    driver.back()
    return user_ids

    
haseenabyashna_follower_ids = get_user_ids("followers","haseenabyashna")
time.sleep(3)
sodelhi_follower_ids = get_user_ids("followers","allthefeels.official")
time.sleep(3)
my_followers = get_user_ids("followers","sample")
time.sleep(3)
i_am_following = get_user_ids("following","sample")


time.sleep(3)
# 500 followers of haseenabyashna
print(haseenabyashna_follower_ids)

# 500 followers of allthefeels
print(allthefeels_follower_ids)


# Print follower of allthefeels who are being followed by me and they aren't following back
for user_id in i_am_following:
    if user_id in haseenabyashna_follower_ids and user_id not in my_followers:
        print(user_id)